## Sequence GAN from char-rnn
This is a character-level language model using recurrent neural networks based Sequence GAN (SeqGAN).
SeqGAN was proposed to cover discrete sequence data.
In this assignment, you will implement SeqGAN with shakespeare data used in assignment 3.

Original blog post & code:
https://github.com/LantaoYu/SeqGAN

That said, you are allowed to copy paste the codes from the original repo with an additional effort to apply it to our data.
HOWEVER, try to implement the model yourself first, and consider the original source code as a last resort.
You will learn a lot while wrapping around your head during the implementation. And you will understand more clearly in a code level.

### AND MOST IMPORTANTLY, IF YOU JUST BLINDLY COPY PASTE THE CODE, YOU SHALL RUIN YOUR EXAM.
### The exam is designed to be solvable for students that actually have written the code themselves.
At least strictly re-type the codes from the original repo line-by-line, and understand what each line means thoroughly.

## YOU HAVE BEEN WARNED.

Now proceed to the code. You may use textloader in previous assingment or not. You can 
freely create another python files (\*.py) and then import them. Following codes can be modified as you want. Just make sure that SeqGAN training works.



In [1]:
from __future__ import print_function
# ipython magic function for limiting the gpu to be seen for tensorflow
# if you have just 1 GPU, specify the value to 0
# if you have multiple GPUs (nut) and want to specify which GPU to use, specify this value to 0 or 1 or etc.
%env CUDA_DEVICE_ORDER = PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES = 0
# load a bunch of libraries

import tensorflow as tf
from tensorflow.contrib import rnn
from tensorflow.contrib import legacy_seq2seq
import numpy as np
import argparse
import time
import os
from six.moves import cPickle
from six import text_type
import sys

# this module is from the .py file of this folder
# it handles loading texts to digits (aka. tokens) which are recognizable for the model
from utils import TextLoader

# for TensorFlow vram efficiency: if this is not specified, the model hogs all the VRAM even if it's not necessary
# bad & greedy TF! but it has a reason for this design choice FWIW, try googling it if interested
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

data_dir = 'data/tinyshakespeare'
seq_length = 20
batch_size = 32

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0


Write down the generator class and any other methods required for the generator class. You may define such methods in other python files (ex : utils.py).

In [2]:
from generator import Generator

Write down the discriminator class and any other methods required for the discriminator class. You may define such methods in other python files (ex : utils.py).

In [3]:
from discriminator import Discriminator

If you need any other class or method, use below blanks. You may insert or delete blanks as many you want. Of course, you may define them in other python files and then import them.

In [4]:
import time
import os

import tensorflow as tf	
import numpy as np

from rollout import ROLLOUT

from utils import TextLoader

In [5]:
class SampleLoader():
	def __init__(self, pos_loader, generator, generated_num, batch_size, sess):
		self.batch_size = batch_size

		positive_examples = []
		negative_examples = []

		# Positive Samples
		pos_loader.reset_batch_pointer()
		for _ in range(pos_loader.num_batches):
			x, _ = pos_loader.next_batch()
			positive_examples.extend(x)

		# Generate Samples
		for _ in range(int(generated_num / batch_size)):
			negative_examples.extend(generator.generate(sess))

		self.sentences = np.array(positive_examples + negative_examples)
		
		positive_labels = [[0, 1] for _ in positive_examples]
		negative_labels = [[1, 0] for _ in negative_examples]
		self.labels = np.concatenate([positive_labels, negative_labels], 0)

		# Shuffle the data
		shuffle_indices = np.random.permutation(np.arange(len(self.labels)))
		self.sentences = self.sentences[shuffle_indices]
		self.labels = self.labels[shuffle_indices]

		# Split batches
		self.num_batch = int(len(self.labels) / self.batch_size)
		self.sentences = self.sentences[:self.num_batch * self.batch_size]
		self.labels = self.labels[:self.num_batch * self.batch_size]
		self.sentences_batches = np.split(self.sentences, self.num_batch, 0)
		self.labels_batches = np.split(self.labels, self.num_batch, 0)
	
		self.pointer = 0

	def next_batch(self):
		ret = self.sentences_batches[self.pointer], self.labels_batches[self.pointer]
		self.pointer = (self.pointer + 1) % self.num_batch
		return ret

	def reset_pointer(self):
		self.pointer = 0

Write down the main code processing the train. You should show at least 16 generated text sequences at the end of the training. We will judge your progress with your final generated result. Be sure to pretrain the generator using supervised frame before training the model with SeqGAN framework.

In [6]:
data_dir = 'data/tinyshakespeare'
save_dir = 'models_seqgan'

###
BATCH_SIZE = 128
SEQ_LENGTH = 100
EMB_DIM = 96
HIDDEN_DIM = 86
START_TOKEN = 0
PRE_EPOCH_NUM = 150
#PRE_EPOCH_NUM = 1

###
dis_epoch_num = 80
#dis_epoch_num = 1
dis_embedding_dim = 128
dis_filter_sizes = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20]
dis_num_filters = [100, 200, 200, 200, 200, 100, 100, 100, 100, 100, 160, 160]
dis_dropout_keep_prob = 0.75
dis_l2_reg_lambda = 0.2
dis_batch_size = 128

###
TOTAL_BATCH = 100
#TOTAL_BATCH = 1
positive_file = 'save/real_data.txt'
negative_file = 'save/generator_sample.txt'
eval_file = 'save/eval_file.txt'
generated_num = 3000

In [7]:
def train():
	###
	if os.path.exists(save_dir):
		os.system('rm -rf {}'.format(save_dir))

	###
	data_loader = TextLoader(data_dir, BATCH_SIZE, SEQ_LENGTH)
	vocab_size = data_loader.vocab_size

	###
	config = tf.ConfigProto()
	config.gpu_options.allow_growth = True
	tf.reset_default_graph()

	###
	generator = Generator(vocab_size, BATCH_SIZE, EMB_DIM, HIDDEN_DIM, SEQ_LENGTH, START_TOKEN)
	discriminator = Discriminator(sequence_length=SEQ_LENGTH, num_classes=2, vocab_size=vocab_size, embedding_size=dis_embedding_dim, 
			filter_sizes=dis_filter_sizes, num_filters=dis_num_filters, l2_reg_lambda=dis_l2_reg_lambda)

	###
	sess = tf.Session(config=config)
	sess.run(tf.global_variables_initializer())
	saver = tf.train.Saver(tf.global_variables())

	####
	for epoch in range(PRE_EPOCH_NUM):
		supervised_g_losses = []
		data_loader.reset_batch_pointer()
		for _ in range(data_loader.num_batches):
			x, _ = data_loader.next_batch()
			_, g_loss = generator.pretrain_step(sess, x)
			supervised_g_losses.append(g_loss)
		loss = np.mean(supervised_g_losses)
		if epoch % 10 == 0:
			print('Generator pretrain : epoch {} - loss {:.4f}'.format(epoch, loss))

	####
	for epoch in range(dis_epoch_num):
		dis_data_loader = SampleLoader(data_loader, generator, generated_num, BATCH_SIZE, sess)
		for _ in range(3):
			dis_data_loader.reset_pointer()
			for it in range(dis_data_loader.num_batch):
				x_batch, y_batch = dis_data_loader.next_batch()
				feed = {
					discriminator.input_x: x_batch,
					discriminator.input_y: y_batch,
					discriminator.dropout_keep_prob: dis_dropout_keep_prob
				}
				_ = sess.run(discriminator.train_op, feed)
		if epoch % 10 == 0:
			print('Discriminator pretrain : epoch {}'.format(epoch))

	rollout = ROLLOUT(generator, 0.8)

	####
	for total_batch in range(TOTAL_BATCH):
		start_time = time.time()
		print('Adversal train : batch {} : START'.format(total_batch))
		# Train the generator for one step
		for it in range(1):
			samples = generator.generate(sess)
			rewards = rollout.get_reward(sess, samples, 16, discriminator)
			feed = {generator.x: samples, generator.rewards: rewards}
			_ = sess.run(generator.g_updates, feed_dict=feed)

		# Update roll-out parameters
		rollout.update_params()

		# Train the discriminator
		for _ in range(5):
			dis_data_loader = SampleLoader(data_loader, generator, generated_num, BATCH_SIZE, sess)
			for _ in range(3):
				dis_data_loader.reset_pointer()
				for it in range(dis_data_loader.num_batch):
					x_batch, y_batch = dis_data_loader.next_batch()
					feed = {
						discriminator.input_x: x_batch,
						discriminator.input_y: y_batch,
						discriminator.dropout_keep_prob: dis_dropout_keep_prob
					}
					_ = sess.run(discriminator.train_op, feed)

		if total_batch % 1 == 0:
			samples = generator.generate(sess)
			for i in range(20):
				print(''.join([data_loader.chars[sample] for sample in samples[i]]))

		if total_batch % 1 == 0:
			checkpoint_path = os.path.join(save_dir, 'model.ckpt')
			saver.save(sess, checkpoint_path, global_step=total_batch)
			print("model saved to {}".format(checkpoint_path))

		print('Adversal train : batch {} : END tooks {}'.format(total_batch, time.time() - start_time))

In [11]:
def test():
	###
	data_loader = TextLoader(data_dir, BATCH_SIZE, SEQ_LENGTH)
	vocab_size = data_loader.vocab_size

	###
	config = tf.ConfigProto()
	config.gpu_options.allow_growth = True
	tf.reset_default_graph()

	###
	generator = Generator(vocab_size, BATCH_SIZE, EMB_DIM, HIDDEN_DIM, SEQ_LENGTH, START_TOKEN)
	discriminator = Discriminator(sequence_length=SEQ_LENGTH, num_classes=2, vocab_size=vocab_size, embedding_size=dis_embedding_dim, 
			filter_sizes=dis_filter_sizes, num_filters=dis_num_filters, l2_reg_lambda=dis_l2_reg_lambda)

	###
	with tf.Session(config=config) as sess:
		sess.run(tf.global_variables_initializer())
		saver = tf.train.Saver(tf.global_variables())
		ckpt = tf.train.get_checkpoint_state(save_dir)
		if ckpt and ckpt.model_checkpoint_path:
			saver.restore(sess, ckpt.model_checkpoint_path)

		samples = generator.generate(sess)
		for i in range(16):
			print(''.join([data_loader.chars[sample] for sample in samples[i]]))

In [9]:
train()

loading preprocessed files
Generator pretrain : epoch 0 - loss 2.5332
Generator pretrain : epoch 10 - loss 1.5599
Generator pretrain : epoch 20 - loss 1.5012
Generator pretrain : epoch 30 - loss 1.4782
Generator pretrain : epoch 40 - loss 1.4659
Generator pretrain : epoch 50 - loss 1.4593
Generator pretrain : epoch 60 - loss 1.4555
Generator pretrain : epoch 70 - loss 1.4495
Generator pretrain : epoch 80 - loss 1.4498
Generator pretrain : epoch 90 - loss 1.4458
Generator pretrain : epoch 100 - loss 1.4439
Generator pretrain : epoch 110 - loss 1.4463
Generator pretrain : epoch 120 - loss 1.4425
Generator pretrain : epoch 130 - loss 1.4434
Generator pretrain : epoch 140 - loss 1.4397
Discriminator pretrain : epoch 0
Discriminator pretrain : epoch 10
Discriminator pretrain : epoch 20
Discriminator pretrain : epoch 30
Discriminator pretrain : epoch 40
Discriminator pretrain : epoch 50
Discriminator pretrain : epoch 60
Discriminator pretrain : epoch 70
Adversal train : batch 0 : START
n suc

model saved to models_seqgan/model.ckpt
Adversal train : batch 3 : END tooks 154.0967664718628
Adversal train : batch 4 : START
s a hear you in my day is't bury majorce will honour were your all-give you.

MISTRENLO:
He, make gu
 remands!

KING RICHARD II:
Houghs wish him worse these allies, a hords. Is most go then, give my go
a man.

Sice was come to.
Postius gonglone, you hisleamary mind of thingd a mores dognkyss love: dag
d my, fooly by dead, or wills: a magacoam, to my more they not, to herely.

PRINCE:
I sureness the d
paken again, will would do your change?

Lest.

KI WATERMEN:
What is dutly majusing pastion: when he
an am generous allay! that holved. Dear,
Nor wish noble applence lomblably slannish noble not resorr
 yo, gives,--on all my good ellue.

PETOLASS:
A majuyorly a mounta.
Restences been quore, well slann
y, I to hot promislaids.

LAPUTIUS:
King Lorrolly: a might in the mineth!

Lastr:
Dise on exedlige m
glated got genmerness'd montane been he my week I amables, my go

model saved to models_seqgan/model.ckpt
Adversal train : batch 7 : END tooks 154.38939547538757
Adversal train : batch 8 : START
theservets me heravecomidy.

KIPCENIO:
For, they theye, quie.

Gods.:
Peak.

KI Cistamenss,
Is meet.
all, they, will I:
Why, getreme me.

LATLEB:
Do, were me: they, faret, to young yith! and glosed,
If
nth I

KING LIFFORCON:
Be; O, my cheramested miesly blusleably winteth.

KING RICK:
I moberenim die 
aves, any a mage allged: gunement!

LATENLE:
Where a so:
'Histy me, 'tis every.

KING LIO:
I he don'
 me, when Camillatestesed; lorg.

KIPCA:
What or.

DERINCIO:
Ay, wable: all'd you, thank, my thath y
 evellel.

KI that I:
Siss.

KING LICHARD:

TRARY:
I ye! ally Abongs!

KING RICK:
I dreeforesh of is
ds.

KING RICHARD II:
I am mine.

CLAUDIO:
Speaks.

KIPCENTIO:
Looks of winth mean, they, mewall nev
e!

KING RICK:
I cadessible-likes every lusty thoughs,--that purpose in lifes.

KING RICK:

KING:
Wh
d, that my done.

KING LICK:
Unmipe eyes, passed that yours ins

model saved to models_seqgan/model.ckpt
Adversal train : batch 11 : END tooks 155.03798866271973
Adversal train : batch 12 : START
gatuply wellsongeal she a came him.

KING RICHARD II:
It olstiresshss up.

LATHBURY:
I muntripall up
the execy,
gifftely:
Gor I mee:
I ye! I neguestees bear,
I am
Thy, they, ulds, Lullisage ele youryin
ake younget.

KING RICHARD II:
Issellsecly Dountestlesion me?

KIPCENTIO:
Dealty theye she,
Is eye?

 yone any cors of am Aflasts!

KIPALIN:
Respitly they.

KING RICHARD II:
What, if you up the the nee
there you:
Let
My lorgesserel elsees.

LATHBURY:
In metcheable usome.

KING RICHARD II:
Stanspate: t
y!

KING RICHARD II:
If
Bule.

KING,
If I mean the art:
An hell majuressedels: a me.

Serretley,
On 
y,
When they, the mamemang my desperbly!

RICING RICHARID:
Whele;
In youreale,
after:
I ye!

Gacesel

geneserage abmenty;
Griens;
Fox; honourapt of heremented,
One
my leaved eatels; needstulrspripestab
ex, a mine: hore!

KING RICHARD II:
Gee, nor arese,
Anth the 

model saved to models_seqgan/model.ckpt
Adversal train : batch 15 : END tooks 154.72877669334412
Adversal train : batch 16 : START
lds.

RICINCUS:
King.

QUEENET:
Aflyor:
Anorayy;
Wieedly.

METRESIO:
In my my heery;
Seem you you Le
ilstup the, thing me, queralisks,
I am generessed of empirior?

KING HENRY Belff
A thing the listrel
have me!

KIPPARD:

LENTIOND:

GLOUCESCFERLAED:
I dream the shortchertry.

KING RICHARD II:
Digged t
o, you,
Readpery, thy.

Firsth Best.

LATENLE:
Fear the exelkn my Trust,
Deken.

KIPPARWARDONT:
Athe
thy the lid, think you.

KING HENRY BOLY:
They, cheeper!

AUFIET:
If my sh bleatup,
If the thy neet;
there the;
Pronaly,
I the mesh
Whe lombly is will yet the the the mee my came my chign histable up, 
then you,
The,
With reversed the hear never, they the myse,
If retpiretupt me the a greeth.

BUCKING
vesh my love the chestle me,
The the the earts,
the hereing the the eally upore mubline, the the thi
the miseriessed Is;
Upot spring of
yielly,
Petriftelit,
O, th

...

model saved to models_seqgan/model.ckpt
Adversal train : batch 83 : END tooks 155.20874214172363
Adversal train : batch 84 : START
 no, stake!

POLTHT:
Atly Bosst; heart; hence a hes thiot poomy the, the repos my a smithe! Go, no, 
h, hep, Oxfor:

Glood,
Ah, you, queence, the, Bumbuny, no, Go, it?

Glord,
Of youthe!

Glofford,

Bo
the whes heed, oftly, the he;
DUWICHAdL:

Yo:
'TnOFkis the he, palians of who oftle!'st heagh, she w
o!

POLTHTHAGAl Go,
'I knee e'et, bedrpither's moen, oft, quees; concealy!

Glood,
An hourwedn, quee
iskned stea heshall'n st opese.

PAULII:
Anyista, no.

Glop; thire yound nor he's nor he
theY yo; th
h, the hourtled, queench; niguthing queenna at thimollam, offry, headed more the know, the! whithe
c
e ot? o' pawnetuse! thind he

The!

Glop,
The, quartorknece poefumply, gitmed hist
The.

POMPHAUM:
G
the!

POLTHTTUSRRSUMNENCSONIO:

Glop, spick?

Boy:
'THOW GRebece, the! noo no, Go, queen thef she
'A
h
Wowh of the who his herd ot, hevoesluck.

Glockleve,
Do of 

model saved to models_seqgan/model.ckpt
Adversal train : batch 87 : END tooks 154.47009706497192
Adversal train : batch 88 : START
thie, Go, a God, mounch heshen?

Glood,' quee,
qook you, thee, otfould hevise?

POLTHTHAS:
I hear
Bo
hinkes abrusly ot.

Glood,
't houratu'st.

POLTHTISGE:
Ho chist,
Lads hes hes Go, the he, no, hep mo
h, got; no, Ofe!

Glood,
At your your ea, queest;
Ploom,

Glofford; a pooss,
An, a preveeve, parke, 
re! henchard Allop; the orad, here eat not; thempoan queenawented. How?

POTHNG RICHARLAFLOn:
Gowned
er; no herrop? whom, nead ot of it hirot expy God.

POLTHTHAT:
At Horenke, ofed, a wood,
PAUH:
Do do
o mousn.

Glood,
' when, the hourtising you, a Gackes mece pas hink oft; the?

POLTHTA:
At Gengh otf
ot: easle! never, thep, pallkinge!

POFHOTHEN:
Send the?

Pagg, fackinge,
preveal! a pads menet; no,
p othef, OxfooR:

Yr:

Goorf,
Mond, quees; quee no: theY queen't; near lorg, quind, the, quees, no m
the't thourmy queen; here the yoo yong hes no, hispoes heY'st

model saved to models_seqgan/model.ckpt
Adversal train : batch 91 : END tooks 154.8200445175171
Adversal train : batch 92 : START
mishamoy no?

POTHNGS:
Greed,

Glorgh
she stay heeple, then pheer it nexumened,
Boy, theY a queen; n
thef BUNTENK:
Knodse, baboy me thielknesh mure it;
A a Gods, God; heare mounninge's yond hepreed.

G
they id foos yong Go, othayonsake specestabled.

Shoes
she is showen'tn my heirft.

II:
Moce heve, h
 here showence: a Gownerghted the whese heY hisw mounm, exed, a wap; a Composhesd, thee whes shepeth
then hesre, knomlabho expream; Gownebaning your monssssed, head oft; hesse, theshing mounn thead?

P
nd, Boyg, thee,
'I hes'lmomlang God, hispoesh, head, thepang.

PAUHNT:
An ea hourtnealy!

POMPAUTAS:
o moons shay; theYonge!
O,
Of hom,
O be
DUNIOLIS:
Whe verrus solaba,'spore,
Knoy hist;
At your hespe
e: moest; no, quees, Beap off; a poan, quins heap your hish, no, Go;
Hea he?

POMPATHAGLANLIO:
'AThi
thef doe, BURHENCA:
Ay; heirfole poop thess' moee's expeald.



model saved to models_seqgan/model.ckpt
Adversal train : batch 95 : END tooks 155.73507142066956
Adversal train : batch 96 : START
thet whe knof; thebud; heare ustervaske moener, mouns oft.

MENCGENTLO:
Enmy:
Food's peved, God, hir
eebernly betarjess heed hist, mooes, otay Lerreno:
At youn-ea, moenaloislusk.

POLTHTHAGRALLO:
Com,

he!

BOLTION:
Go,

Awixe
henge, hes whe hoursed, no, Gowneed their heedsf, thee, heir's heir dreed o
he, no, Oirst so ffom, no, he, the! who yot, hes his heply yonged thoen here, sowenne,
't; 't; no, G
 nothe!

POMPSHARCUS:

Glorgmed, here no,

POMPSHEO:
Go, otan herebake,
Ataye, queenced, head here, 
thie op;
FoOd IIIUPERCA:
Go, shoee she shouke,
Do your wask; theppot;
Shoffed, hep houchast,
'I:

Ah
IOLUS:
The, a thee, ot,
Of yot.

BUCKIA:
An, here queented, thep, whe.

POLTHTHAY:
If imejes;
Kome,

NTOS:
KnOMENENENO:

Glorgh,

ET:

Boy:

POMPAUTOND:
Their.

Boy: shouns;
Go, toom whe, genemon of yo
h queentnessed.

POMPSHE:
At it,
Boy?

GONZ:
'bad whe's whoee

model saved to models_seqgan/model.ckpt
Adversal train : batch 99 : END tooks 155.8518602848053


In [18]:
test()

loading preprocessed files
INFO:tensorflow:Restoring parameters from models_seqgan/model.ckpt-99
then, a fourmeather, the' you's empoop,
Rooknam, it:
' whoen oft; nof, here, hepledd of oft;
Angrean
te, noe, ot sheppostore.

III.

POLTHTHASARS:
If Gook;
Whas whey thinee histalugbua, be shreact; the
thie, them; shepe hearish; no; no mornemerin's oft she hear'd so sle?

Glord.

HOMESS:
At choen?
It 


POMPETO:

POTHNGARCUS:
DO OGH Goe!

POLTHTHAGRALOT:
Go;

Groog Towelishband;
Boysthead mesporieato
Ad: Athoo loes' moosce heYes?

JUNIETO:
Horfake's imp here!

Pacch;
An!

POLTHTHMENCH:
Thio st; here
, a moan us; a pood hes sheppesnast; no noY queens here theer ches held hes he, Beoubain!

GONZIOS:

a be'sted, otfowh hish, thee hes' chappake oft;
Of Nood;
Un? Hores here,
A moan, heboy tooly fourd, 
ng loachal, Boy?

JUNCHUSUMBOLING RICHO:
So;
Then hest
At hinchard hes,
Away, thepain' pay;
Kave!

B
e, heed; nou'e now;
Pows;
Ox
Do;

Gred
Se

Peacus;
The, goot.

Greack he hearfh, a ege.

IIIIOG